#  Table detection using a CNN

In [1]:
%matplotlib inline
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import random

## Generate data

In [2]:
def generateData(numImages=1,pageSize=(100,100),circleSize=10):
    images = []
    images1D = np.zeros((numImages,pageSize[0]*pageSize[1]),dtype=np.float32)
    labels = []

    for i in range(numImages):
        page = np.full(pageSize, 255, dtype=np.uint8)

        location = (circleSize+int(random.random()*(pageSize[1]-circleSize*2)), circleSize+int(random.random()*(pageSize[0]-circleSize*2)))
        cv2.circle(page, location,circleSize, (0, 0, 255))

        images.append(page)
        images1D[i] = page.reshape(pageSize[0]*pageSize[1])
        labels.append((location))
    images1D /= 255.0
    return images, labels,images1D

In [3]:
circleSize = 6
pageSize = (28, 28)

images, labels,images1D = generateData(numImages=1000,pageSize=pageSize,circleSize=circleSize) #Do not make this more that around 1000

trainImages = images1D[:800]
trainImagesD = images[:800]

testImages = images1D[800:]
testImagesD = images[800:]

trainLabels = labels[:800]
testLabels = labels[800:]

## Display the data (Causes kernel restart)

In [4]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [5]:
x = tf.placeholder(tf.float32, shape=[None, pageSize[0]*pageSize[1]])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

## Weight Initialization

To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons". Instead of doing this repeatedly while we build the model, let's create two handy functions to do it for us.

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Convolution and Pooling

TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

In [7]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## First convolutional layer

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolutional will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32]) #(window_dim1,window_dim1,input channels,output channels)
b_conv1 = bias_variable([32])

To apply the layer, we first reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

The first dimension is specified as -1 since it will be inferred from the number of images in the batch.

In [9]:
x_image = tf.reshape(x, [-1,pageSize[0],pageSize[1],1])

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.

In [10]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## Second Convolutional Layer



In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

In [11]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [12]:
size = 7*7*64
W_fc1 = weight_variable([size, 1024])#Was 7*7*64
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, size]) #Was 7*7*64
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.

In [13]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## Readout Layer

Finally, we add a softmax layer, just like for the one layer softmax regression above.

In [14]:
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv=tf.matmul(h_fc1_drop, W_fc2) + b_fc2#tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## Train and Evaluate the Model

How well does this model do? To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network.

The differences are that:

We will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer.

We will include the additional parameter keep_prob in feed_dict to control the dropout rate.

We will add logging to every 100th iteration in the training process.

Feel free to go ahead and run this code, but it does 20,000 training iterations and may take a while (possibly up to half an hour), depending on your processor.



In [ ]:
squareError = tf.reduce_sum(tf.pow(y_conv - y_, 2))
train_step = tf.train.AdamOptimizer(1e-4).minimize(squareError)
accuracy = tf.reduce_mean(tf.abs(y_conv - y_))

sess.run(tf.initialize_all_variables())

batchSize = 20

numBatches = len(trainImages)/batchSize


for i in range(600):

  lowerLimit = (i%numBatches)*batchSize
  upperLimit = (i%numBatches+1)*batchSize

  imageBatch = trainImages[lowerLimit:upperLimit]
  labelBatch = trainLabels[lowerLimit:upperLimit]
  
  if i%100 == 0:
    #print imageBatch[0]
    #print len(imageBatch[0])
    #print(len(labelBatch))
    train_accuracy = accuracy.eval(feed_dict={x:imageBatch, y_: labelBatch, keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: imageBatch, y_: labelBatch, keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: testImages, y_: testLabels, keep_prob: 1.0}))

step 0, training accuracy 11.597
step 100, training accuracy 0.797711
step 200, training accuracy 0.607553
step 300, training accuracy 0.38127
step 400, training accuracy 0.434791
step 500, training accuracy 0.476604
test accuracy 0.445077


## Evaluate the network using the testing data

In [ ]:
for i in range(len(testImages)):
    testImage = testImagesD[i].copy()
    image = testImages[i].reshape((1,784))

    #Get the estimate center
    estimate = sess.run([y_conv],feed_dict={x:image, keep_prob: 1.0})
    #print y_conv
    
    label = testLabels[i]
    estimate = estimate[0][0]
    print "True center: ", label, ". Estimated center: ", estimate
    
    cv2.rectangle(testImage, (int(estimate[0])-circleSize, int(estimate[1])-circleSize), (int(estimate[0])+circleSize, int(estimate[1])+circleSize), (0, 0, 0), 1)

    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    cv2.imshow("Image", testImage)
    cv2.waitKey(0)
cv2.destroyWindow()

True center:  (11, 17) . Estimated center:  [ 11.47093964  16.85348511]
True center:  (18, 9) . Estimated center:  [ 17.71611786   8.52111816]
True center:  (18, 10) . Estimated center:  [ 17.94909286   9.24249935]
True center:  (20, 16) . Estimated center:  [ 19.79645348  14.65106583]
True center:  (20, 12) . Estimated center:  [ 19.54772568  11.41810322]
True center:  (13, 13) . Estimated center:  [ 12.55906677  12.44930935]
True center:  (17, 12) . Estimated center:  [ 16.19859123  11.47110939]
True center:  (16, 7) . Estimated center:  [ 15.25411606   6.64921141]
True center:  (9, 6) . Estimated center:  [ 9.69881248  6.30195951]
True center:  (12, 10) . Estimated center:  [ 12.15592384   9.88041687]
True center:  (6, 16) . Estimated center:  [  6.58792639  15.25544453]
True center:  (11, 11) . Estimated center:  [ 10.7983942   10.95752716]
True center:  (9, 17) . Estimated center:  [  8.90152931  17.1405201 ]
True center:  (16, 16) . Estimated center:  [ 15.67473984  15.36889076]


True center:  (19, 12) . Estimated center:  [ 18.63262749  11.75087738]
True center:  (20, 9) . Estimated center:  [ 19.1490078    8.15168285]
True center:  (7, 12) . Estimated center:  [  7.09551239  11.06694698]
